In [1]:
from src.lexer.tokens import Token, TokenType

token = Token(type=TokenType.INTEGER, value="123", line=1, column=1)
print(token)

Token(type=<TokenType.INTEGER>, value='123', line=1, column=1)
